In [ ]:
import datetime

today = datetime.date.today()
yesterday = today - datetime.timedelta(1)
target = "{}-{}".format(yesterday.month, yesterday.day)

f = open("./per_day_keyword/(keyword)" + target + ".txt", mode = "r", encoding="utf-8")
text = f.read()
exec("text = " + text)
f.close()

from collections import Counter
count = Counter(text)
keywords = count.most_common(50)

# 키워드, 개수, 하루 총 단어 수 저장
keys = [ item[0] for item in keywords ]
counts = [ item[1] for item in keywords ]

# WordCloud 그래프
import matplotlib.pyplot as plt
from wordcloud import WordCloud

font_path = "./malgunbd.ttf"

wc = WordCloud(font_path=font_path, background_color='white', width=1200, height=600)
cloud = wc.generate_from_frequencies(dict(keywords))
plt.figure(figsize=(15,8))
plt.axis('off')
plt.imshow(cloud)
plt.savefig("./graph/" + target + "-b.jpg")

# 막대그래프
import numpy as np
from matplotlib import font_manager, rc
from matplotlib import style

font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
style.use('ggplot')

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(111)

pos = np.arange(50)
rects = plt.bar(pos, counts[0:50], align='center', width=0.5)
plt.xticks(pos, keys)

for i, rect in enumerate(rects):
    ax.text(rect.get_x() + rect.get_width() / 2.0, 0.95 * rect.get_height(), str(counts[i]), ha='center')
    
plt.xlabel('\n키워드')
plt.ylabel('언급횟수\n')
plt.savefig("./graph/" + target + "-a.jpg")
# plt.show()


# 업로드
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))
    
bucket_name = 'gunmin_test'
source_file_name = "./graph/" + target + "-a.jpg"
destination_blob_name = "graph/" + target + "-a.jpg"

upload_blob(bucket_name, source_file_name, destination_blob_name)

bucket_name = 'gunmin_test'
source_file_name = "./graph/" + target + "-b.jpg"
destination_blob_name = "graph/" + target + "-b.jpg"

upload_blob(bucket_name, source_file_name, destination_blob_name)